In [1]:
from multiprocessing import Process
import time
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import re
import pickle
from scipy import spatial

import requests
import json
import argparse


def get_jaccard_sim(str1, str2):
    a = set(str1.split())
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


def euclidean_distance(a, b):
    dist = np.linalg.norm(a-b)
    return dist


def cosine_distance(a, b):
    result = 1 - spatial.distance.cosine(a, b)
    return result

# get_jaccard_sim("điện thoại vsmart joy 4" ,"bo mạch chủ main msi mag b460m mortar")

In [2]:
df = pd.read_pickle("./df.pkl")
name_embeding = np.load("name_embeding.npy", allow_pickle=True)
# name_embeding

In [3]:
import time

def creat_graph(id):
    time_start = time.time()
    print("doing ./pair_id/pair_id_"+str(id)+".pkl")
    df_pair_id = pd.read_pickle("./pair_id/pair_id_"+str(id)+".pkl")
    
    arr_obj = []
    count_arr = 0
    for i in df_pair_id:
        df_item_from = df[df["id"] == i[0]]
        df_item_to = df[df["id"] == i[1]]
        
        name_embeding_id_from = np.where(
            (name_embeding[:, 0] == i[0]))[0][0]
        a = name_embeding[name_embeding_id_from][1][0]

        name_embeding_id_to = np.where((name_embeding[:, 0] == i[1]))[0][0]
        b = name_embeding[name_embeding_id_to][1][0]

        arr_obj.append({
            "from_id": i[0],
            "to_id": i[1],
            "get_jaccard_sim":float(get_jaccard_sim(df_item_from["name"].values[0],df_item_to["name"].values[0])),
            "euclidean_distance": float(euclidean_distance(a, b)),
            "cosine_distance": float(cosine_distance(a, b)),
            "same_category": bool((df_item_from["productset_group_name"].values == df_item_to["productset_group_name"].values)[0]),
            "same_brand_name": bool((df_item_from["brand.id"].values == df_item_to["brand.id"].values)[0]),
            "same_current_seller": bool((df_item_from["current_seller.id"].values == df_item_to["current_seller.id"].values)[0]),
            "more_expensive": bool((df_item_from["price"].values - df_item_to["price"].values)[0]),
            "more_discount": bool((df_item_from["discount"].values - df_item_to["discount"].values)[0]),
            "more_discount_rate": bool((df_item_from["discount_rate"].values - df_item_to["discount_rate"].values)[0]),
            "more_price_segment": int((df_item_from["price_segment"].values - df_item_to["price_segment"].values)[0]),
            "more_rating": int((df_item_from["rating_average"].values - df_item_to["rating_average"].values)[0]),
            "older": int((df_item_from["day_ago_created"].values - df_item_to["day_ago_created"].values)[0]),
            "more_review_count": int((df_item_from["review_count"].values - df_item_to["review_count"].values)[0]),
        })
        
        if len(arr_obj)==10000:
            arr_obj = np.array(arr_obj)
            with open('./data_graph/pair_id_'+str(id)+'_'+str(count_arr)+'.npy', 'wb') as f:
                np.save(f, arr_obj)
            print("saving ",id, count_arr,"time exc: ",time.time()-time_start)
            arr_obj = []
            count_arr = count_arr + 1

In [ ]:
# ids = [0, 1, 2,3,4,5,6,7,8,]
# groups = np.arange(375)
# groups = groups.reshape(15,25)
group = [375,376]

procs = []

# for group in groups:
for id in group:
    proc = Process(target=creat_graph, args=(id,))
    procs.append(proc)
    proc.start()
for proc in procs:
    proc.join()

doing ./pair_id/pair_id_375.pkl
doing ./pair_id/pair_id_376.pkl
saving  375 0 time exc:  28.85222840309143
saving  376 0 time exc:  29.450605869293213
saving  376 1 time exc:  58.00700807571411
saving  375 1 time exc:  59.243396043777466
saving  376 2 time exc:  86.15284037590027
saving  375 2 time exc:  89.40891885757446
saving  376 3 time exc:  115.24127674102783
saving  375 3 time exc:  119.39200019836426
saving  376 4 time exc:  143.94796538352966


In [30]:
from pymongo import MongoClient
from bson.json_util import loads, dumps
import json

myclient = MongoClient("mongodb://112.127.129.117:27017/")

mydb = myclient["tiki"]


def db_col_data_graph():
    return mydb["data_graph"]
myclient

MongoClient(host=['112.127.129.117:27017'], document_class=dict, tz_aware=False, connect=True)

In [31]:
db_col_data_graph().insert_one({"test":"test"})

KeyboardInterrupt: 